In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models, transforms
import torch.nn.functional as F
from BcomMEG import *
from MEGDataset_Conv import *
# from ConvNet import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [22]:
dir = '/Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT'
picks = None
data = BcomMEG(subjects=['BCOM_18_2', 'BCOM_18_3', 'BCOM_18_4', 'BCOM_06_2', 'BCOM_06_3', 'BCOM_06_4', 'BCOM_19_2', 'BCOM_19_3', 'BCOM_19_4'], dir=dir, picks=picks, avoid_reading=True)
data.upscale(13)
dataset = MEGDataset_Conv(data, label_map='vowel_class_label_map')

Reading /Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT/BCOM_18_2_re_144-epo.fif ...
    Found the data of interest:
        t =    -300.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
8 matching events found
No baseline correction applied
0 projection items activated
Reading /Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT/BCOM_18_2_i_116-epo.fif ...
    Found the data of interest:
        t =    -300.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
12 matching events found
No baseline correction applied
0 projection items activated
Reading /Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT/BCOM_18_2_a_112-epo.fif ...
    Found the data of interest:
        t =    -300.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
16 matching events found
No baseline correct

In [52]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        #Conv blocks
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1) #(16, 247, 241)
        self.bn1 = nn.BatchNorm2d(16) #idea from Defossez (King) et al
        self.relu1 = nn.GELU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) #(16, 123, 120)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1) #(32, 123, 120)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.GELU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) #(32, 61, 60)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) # (64, 61, 60)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.GELU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) #(64, 30, 30)

        #fc layers
        self.fc1 = nn.Linear(57600, 512)
        self.fc2 = nn.Linear(512, 18)
        

    def forward(self, x):
        # (1, 247, 241)
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        #print("After conv1:", x.shape)
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        #print("After conv2:", x.shape)
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        #print("After conv3:", x.shape)

        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [71]:
from sklearn.preprocessing import StandardScaler
import copy
scaler = StandardScaler()
dataset_copy = copy.deepcopy(dataset)

dataset_copy.data = scaler.fit_transform(dataset.data.reshape(-1, dataset.data.shape[-1])).reshape(dataset.data.shape)


In [72]:
train_indices, test_indices = train_test_split(
    list(range(len(dataset))),
    test_size=0.3,
    random_state=42,
    stratify=dataset.labels
)
# train_dataset = Subset(dataset, train_indices)
# test_dataset =  Subset(dataset, test_indices)

train_dataset = Subset(dataset_copy, train_indices) 
test_dataset =  Subset(dataset_copy, test_indices)

In [73]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
for batch_index, (data, labels) in enumerate(train_loader):
    print(f"Batch {batch_index}:")
    print(f"  Data shape: {data.shape}")
    print(f"  Labels shape: {labels.shape}")
    break

Batch 0:
  Data shape: torch.Size([64, 1, 247, 241])
  Labels shape: torch.Size([64])


In [74]:
device = 'mps'
model = ConvNet().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

total_steps = len(train_loader)

loss_values = []

epochs = 50
for epoch in range(epochs):
    for i, (data, labels) in enumerate(train_loader):
        model.train()
        data, labels = data.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)

        outputs = model(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_values.append(loss.item())

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch + 1} / {epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}')
print('Training Finished')


Epoch [1 / 50], Step [10/20], Loss: 1.0901
Epoch [1 / 50], Step [20/20], Loss: 1.2427
Epoch [2 / 50], Step [10/20], Loss: 1.0290
Epoch [2 / 50], Step [20/20], Loss: 0.9875
Epoch [3 / 50], Step [10/20], Loss: 0.8877
Epoch [3 / 50], Step [20/20], Loss: 0.8213
Epoch [4 / 50], Step [10/20], Loss: 0.8723
Epoch [4 / 50], Step [20/20], Loss: 0.8253
Epoch [5 / 50], Step [10/20], Loss: 0.7189
Epoch [5 / 50], Step [20/20], Loss: 0.7347
Epoch [6 / 50], Step [10/20], Loss: 0.8224
Epoch [6 / 50], Step [20/20], Loss: 0.6281
Epoch [7 / 50], Step [10/20], Loss: 0.5883
Epoch [7 / 50], Step [20/20], Loss: 0.6965
Epoch [8 / 50], Step [10/20], Loss: 0.5948
Epoch [8 / 50], Step [20/20], Loss: 0.5570
Epoch [9 / 50], Step [10/20], Loss: 0.4524
Epoch [9 / 50], Step [20/20], Loss: 0.6029
Epoch [10 / 50], Step [10/20], Loss: 0.4386
Epoch [10 / 50], Step [20/20], Loss: 0.4968
Epoch [11 / 50], Step [10/20], Loss: 0.4207
Epoch [11 / 50], Step [20/20], Loss: 0.4123
Epoch [12 / 50], Step [10/20], Loss: 0.3549
Epoch 

In [75]:
test_loss_vals = []
test_accuracies = []
all_preds = []
all_labels = []

test_loss = 0
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for data, labels in test_loader:
        data, labels = data.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)

        outputs = model(data)
        loss = criterion(outputs, labels)

        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        total += labels.size(0)
        correct += (predicted ==labels).sum().item()

    test_loss /= len(test_loader)
    accuracy = correct / total

    print(f'Test Loss: {test_loss:.4f}, Accuracy: {100 * accuracy:.2f}%')



Test Loss: 1.5943, Accuracy: 35.90%


In [70]:
from sklearn.metrics import classification_report, confusion_matrix
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))
print("\nDetailed Classification Report:")
print(classification_report(all_labels, all_preds))

# Additional diagnostics
print("\nClass distribution in test set:")
from collections import Counter
print(Counter(all_labels))

# Look at some predictions
print("\nSample predictions vs actual:")
for i in range(min(10, len(all_preds))):
    print(f"Predicted: {all_preds[i]}, Actual: {all_labels[i]}")


Confusion Matrix:
[[57 82 55]
 [63 77 45]
 [45 63 45]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.35      0.29      0.32       194
           1       0.35      0.42      0.38       185
           2       0.31      0.29      0.30       153

    accuracy                           0.34       532
   macro avg       0.33      0.33      0.33       532
weighted avg       0.34      0.34      0.33       532


Class distribution in test set:
Counter({0: 194, 1: 185, 2: 153})

Sample predictions vs actual:
Predicted: 1, Actual: 1
Predicted: 1, Actual: 2
Predicted: 1, Actual: 2
Predicted: 0, Actual: 1
Predicted: 1, Actual: 0
Predicted: 2, Actual: 0
Predicted: 0, Actual: 0
Predicted: 1, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 2


K-Fold

In [ ]:
from sklearn.model_selection import KFold

n_folds = 5
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

train_losses = []
val_losses = []
train_accs = []
val_accs = []